# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 82 new papers today
          35 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/34 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.02671


extracting tarball to tmp_2301.02671...

 done.


Retrieving document from  https://arxiv.org/e-print/2301.02675


...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.03502


extracting tarball to tmp_2301.03502...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.03519


extracting tarball to tmp_2301.03519...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02671-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02671) | **The UNCOVER Survey: A first-look HST+JWST catalog of 50,000 galaxies  near Abell 2744 and beyond**  |
|| John R. Weaver, et al. -- incl., <mark>Natascha Forster Schreiber</mark>, <mark>Anna de Graaff</mark>, <mark>Jenny E. Greene</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *27 pages, 19 figures, submitted to ApJS. Comments welcome!*|
|**Abstract**| In November 2022, the James Webb Space Telescope (JWST) returned deep near-infrared images of Abell~2744 -- a powerful lensing cluster capable of magnifying distant, incipient galaxies beyond it. Together with the existing Hubble Space Telescope (HST) imaging, this publicly available dataset opens a fundamentally new discovery space to understand the remaining mysteries of the formation and evolution of galaxies across cosmic time. In this work, we detect and measure some 50,000 objects across the 45 arcmin$^2$ JWST footprint down to a $5\,\sigma$ limiting magnitude of $\sim$29.9\,mag in 0.32" apertures. Photometry is performed using circular apertures on images matched to the point spread function of the reddest NIRCam band, F444W, and cleaned of bright cluster galaxies and the related intra-cluster light. To give an impression of the photometric performance, we measure photometric redshifts and achieve a $\sigma_{\rm NMAD}\approx0.03$ based on known, but relatively small, spectroscopic samples. With this paper, we publicly release HST and JWST PSF-matched photometric catalogs optimized for bright and extended sources (0.7" apertures) and compact and faint sources (0.32" apertures) along with basic photometric redshifts, rest-frame colors, and individual magnification estimates. These catalogs will set the stage for efficient and deep spectroscopic follow-up of the first JWST-selected samples in Summer 2023. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03209-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03209) | **A JWST NIRSpec phase curve for WASP-121b: dayside emission strongest  eastward of the substellar point and nightside conditions conducive to cloud  formation**  |
|| <mark>Thomas Mikal-Evans</mark>, et al. -- incl., <mark>Jayesh M. Goyal</mark>, <mark>Nikole K. Lewis</mark>, <mark>Duncan A. Christie</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *Accepted for publication in Astrophysical Journal Letters on December 29, 2022*|
|**Abstract**| We present the first exoplanet phase curve measurement made with the JWST NIRSpec instrument, highlighting the exceptional stability of this newly-commissioned observatory for exoplanet climate studies. The target, WASP-121b, is an ultrahot Jupiter with an orbital period of 30.6 hr. We analyze two broadband light curves generated for the NRS1 and NRS2 detectors, covering wavelength ranges of 2.70-3.72 micron and 3.82-5.15 micron, respectively. Both light curves exhibit minimal systematics, with approximately linear drifts in the baseline flux level of 30 ppm/hr (NRS1) and 10 ppm/hr (NRS2). Assuming a simple brightness map for the planet described by a low-order spherical harmonic dipole, our light curve fits suggest that the phase curve peaks coincide with orbital phases $3.36 \pm 0.11$ deg (NRS1) and $2.66 \pm 0.12$ deg (NRS2) prior to mid-eclipse. This is consistent with the strongest dayside emission emanating from eastward of the substellar point. We measure planet-to-star emission ratios of $3,924 \pm 7$ ppm (NRS1) and $4,924 \pm 9$ ppm (NRS2) for the dayside hemisphere, and $136 \pm 8$ ppm (NRS1) and $630 \pm 10$ ppm (NRS2) for the nightside hemisphere. The latter nightside emission ratios translate to planetary brightness temperatures of $926 \pm 12$ K (NRS1) and $1,122 \pm 10$ K (NRS2), which are low enough for a wide range of refractory condensates to form, including enstatite and forsterite. A nightside cloud deck may be blocking emission from deeper, hotter layers of the atmosphere, potentially helping to explain why cloud-free 3D general circulation model simulations systematically over-predict the nightside emission for WASP-121b. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03387-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03387) | **An APEX study of molecular outflows in FUor-type stars**  |
|| Fernando Cruz-Sáenz de Miera, et al. -- incl., <mark>Thomas Henning</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *Accepted for publication in ApJ. 29 pages, 8 tables, 10 figures, 2 figuresets*|
|**Abstract**| FU Orionis-type objects (FUors) are low-mass pre-main-sequence objects which go through a short-lived phase (~100 years) of increased mass accretion rate (from 10^-8 to 10^-4 M_sun yr^-1). These eruptive young stars are in the early stages of stellar evolution and, thus, still deeply embedded in a massive envelope that feeds material to the circumstellar disk that is then accreted onto the star. Some FUors drive molecular outflows, i.e. low-velocity wide-angle magneto-hydrodynamical winds, that inject energy and momentum back to the surrounding envelopes, and help clear the material surrounding the young star. Here we present a 12CO (3--2), 13CO (3--2) and 12CO (4--3) survey of 20 FUor-type eruptive young stars observed with APEX. We use our 13CO (3--2) observations to measure the masses of the envelopes surrounding each FUor and find an agreement with the FUor evolutionary trend found from the 10um silicate feature. We find outflows in 11 FUors, calculate their masses and other kinematic properties, and compare these with those of outflows found around quiescent young stellar objects gathered from the literature. This comparison indicates that outflows in FUors are more massive than outflows in quiescent sources, and that FUor outflows have a higher ratio outflow mass with respect to the envelope than the quiescent sample, indicating that the eruptive young stars have lower star-forming efficiencies. Finally, we found that the outflow forces in FUors are similar to those of quiescent young stellar objects, indicating that their accretion histories are similar or that the FUor outflows have lower velocities. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03502-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03502) | **An FRB Sent Me a DM: Constraining the Electron Column of the Milky Way  Halo with Fast Radio Burst Dispersion Measures from CHIME/FRB**  |
|| Amanda M. Cook, et al. -- incl., <mark>Antonio Herrera-Martin</mark>, <mark>Jane Kaczmarek</mark>, <mark>Bradley W. Meyers</mark>, <mark>Kendrick Smith</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *19 pages, 4 figures. Submitted to AAS Journals on November 22nd, 2022*|
|**Abstract**| The CHIME/FRB project has detected hundreds of fast radio bursts (FRBs), providing an unparalleled population to probe statistically the foreground media that they illuminate. One such foreground medium is the ionized halo of the Milky Way (MW). We estimate the total Galactic electron column density from FRB dispersion measures (DMs) as a function of Galactic latitude using four different estimators, including ones that assume spherical symmetry of the ionized MW halo and ones that imply more latitudinal-variation in density. Our observation-based constraints of the total Galactic DM contribution for $|b|\geq 30^\circ$, depending on the Galactic latitude and selected model, span 87.8 - 141 pc cm^-3. This constraint implies upper limits on the MW halo DM contribution that range over 52-111 pc cm^-3. We discuss the viability of various gas density profiles for the MW halo that have been used to estimate the halo's contribution to DMs of extragalactic sources. Several models overestimate the DM contribution, especially when assuming higher halo gas masses (~ 3.5 x 10^12 solar masses). Some halo models predict a higher MW halo DM contribution than can be supported by our observations unless the effect of feedback is increased within them, highlighting the impact of feedback processes in galaxy formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03519-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03519) | **Evolution of elemental abundances in hot active region cores from  Chandrayaan-2 XSM observations**  |
|| Biswajit Mondal, et al. -- incl., <mark>Helen E. Mason</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *Submitted to the Astrophysical Journal, 16 pages,8 figures*|
|**Abstract**| The First Ionization Potential (FIP) bias, whereby elemental abundances for low FIP elements in different coronal structures vary from their photospheric values and may also vary with time, has been widely studied. In order to study the temporal variation, and to understand the physical mechanisms giving rise to the FIP bias, we have investigated the hot cores of three ARs using disk-integrated soft X-ray spectroscopic observation with the Solar X-ray Monitor (XSM) onboard Chandrayaan-2. Observations for periods when only one AR was present on the solar disk were used so as to ensure that the AR was the principal contributor to the total X-ray intensity. The average values of temperature and EM were ~3 MK and 3.0E46/cm3 respectively. Regardless of the age and activity of the AR, the elemental abundances of the low FIP elements, Al, Mg, and Si were consistently higher than their photospheric values. The average FIP bias for Mg and Si was ~3, whereas the FIP bias for the mid-FIP element, S, was ~1.5. However, the FIP bias for the lowest FIP element, Al, was observed to be higher than 3, which, if real, suggests a dependence of the FIP bias of low FIP elements on their FIP value. Another major result from our analysis is that the FIP bias of these elements is established in within ~10 hours of emergence of the AR and then remains almost constant throughout its lifetime. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02727-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02727) | **Parker Solar Probe: Four Years of Discoveries at Solar Cycle Minimum**  |
|| N. E. Raouafi, et al. -- incl., <mark>K. G. Klein</mark>, <mark>M. Linton</mark>, <mark>P. Hess</mark>, <mark>P. Liewer</mark>, <mark>T. Niembro-Hernandez</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *157 pages, 65 figures*|
|**Abstract**| Launched on 12 Aug. 2018, NASA's Parker Solar Probe had completed 13 of its scheduled 24 orbits around the Sun by Nov. 2022. The mission's primary science goal is to determine the structure and dynamics of the Sun's coronal magnetic field, understand how the solar corona and wind are heated and accelerated, and determine what processes accelerate energetic particles. Parker Solar Probe returned a treasure trove of science data that far exceeded quality, significance, and quantity expectations, leading to a significant number of discoveries reported in nearly 700 peer-reviewed publications. The first four years of the 7-year primary mission duration have been mostly during solar minimum conditions with few major solar events. Starting with orbit 8 (i.e., 28 Apr. 2021), Parker flew through the magnetically dominated corona, i.e., sub-Alfv\'enic solar wind, which is one of the mission's primary objectives. In this paper, we present an overview of the scientific advances made mainly during the first four years of the Parker Solar Probe mission, which go well beyond the three science objectives that are: (1) Trace the flow of energy that heats and accelerates the solar corona and solar wind; (2) Determine the structure and dynamics of the plasma and magnetic fields at the sources of the solar wind; and (3) Explore mechanisms that accelerate and transport energetic particles. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03144-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03144) | **Evidence of high-mass star formation through multi-scale mass accretion  in hub-filament-system clouds**  |
|| <mark>Hong-Li Liu</mark>, et al. -- incl., <mark>Tie Liu</mark>, <mark>Jinhua He</mark>, <mark>Shanghuo Li</mark>, <mark>Chang Won Lee</mark>, <mark>Yong Zhang</mark>, <mark>Xun-Chuan Liu</mark>, <mark>Siju Zhang</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *Accepted for publication in MNRAS; 16 pages, 8 figures, and 3 tables*|
|**Abstract**| We present a statistical study of a sample of 17 hub-filament-system (HFS) clouds of high-mass star formation using high-angular resolution ($\sim$1-2 arcsecond) ALMA 1.3mm and 3mm continuum data. The sample includes 8 infrared (IR)-dark and 9 IR-bright types, which correspond to an evolutionary sequence from the IR-dark to IR-bright stage. The central massive clumps and their associated most massive cores are observed to follow a trend of increasing mass ($M$) and mass surface density ($\Sigma$) with evolution from IR-dark to IR-bright stage. In addition, a mass-segregated cluster of young stellar objects (YSOs) are revealed in both IR-dark and IR-bright HFSs with massive YSOs located in the hub and the population of low-mass YSOs distributed over larger areas. Moreover, outflow feedback in all HFSs are found to escape preferentially through the inter-filamentary diffuse cavities, suggesting that outflows would render a limited effect on the disruption of the HFSs and ongoing high-mass star formation therein. From the above observations, we suggest that high-mass star formation in the HFSs can be described by a multi-scale mass accretion/transfer scenario, from hub-composing filaments through clumps down to cores, that can naturally lead to a mass-segregated cluster of stars. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure 0figs/Mcore_12m_vs_12m7m.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02745-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02745) | **Optical Properties of Organic Hazes in Water-rich Exoplanet Atmospheres:  Implications for Observations with JWST**  |
|| <mark>Chao He</mark>, et al. -- incl., <mark>Nikole K. Lewis</mark> |
|*Appeared on*| *2023-01-10*|
|*Comments*| *4 figures, 1 Table, Paper under review in Nature Astronomy*|
|**Abstract**| JWST has begun its scientific mission, which includes the atmospheric characterization of transiting exoplanets. Some of the first exoplanets to be observed by JWST have equilibrium temperatures below 1000 K, which is a regime where photochemical hazes are expected to form. The optical properties of these hazes, which controls how they interact with light, are critical for interpreting exoplanet observations, but relevant data are not available. Here we measure the optical properties of organic haze analogues generated in water-rich exoplanet atmosphere experiments. We report optical constants (0.4 to 28.6 micron) of organic hazes for current and future observational and modeling efforts covering the entire wavelength range of JWST instrumentation and a large part of Hubble. We use these optical constants to generate hazy model atmospheric spectra. The synthetic spectra show that differences in haze optical constants have a detectable effect on the spectra, impacting our interpretation of exoplanet observations. This study emphasizes the need to investigate the optical properties of hazes formed in different exoplanet atmospheres, and establishes a practical procedure to determine such properties. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.02671.md
    + _build/html/tmp_2301.02671/./figures/dmag_Best-fit_0_32.png
    + _build/html/tmp_2301.02671/figures/detection_depth.png
    + _build/html/tmp_2301.02671/figures/badflag.png
exported in  _build/html/2301.03209.md
    + _build/html/tmp_2301.03209/./fig4_dayside_nightside_t.png
    + _build/html/tmp_2301.03209/./fig3_model_parameters_t.png
    + _build/html/tmp_2301.03209/./fig2_light_curves_t.png
exported in  _build/html/2301.03387.md
    + _build/html/tmp_2301.03387/figures/L1551IRS5_parabolas.png
    + _build/html/tmp_2301.03387/figures/line_profiles.png
    + _build/html/tmp_2301.03387/figures/outflows_32.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1}$
$\newcommand{\arraystretch}{0.9}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1}$
$\newcommand{$\arraystretch$}{0.9}$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.02671-b31b1b.svg)](https://arxiv.org/abs/2301.02671)<mark>Appeared on: 2023-01-10</mark> - _27 pages, 19 figures, submitted to ApJS. Comments welcome!_

</div>
<div id="authors">

John R. Weaver, et al. -- incl., <mark>Natascha Förster Schreiber</mark>, <mark>Anna de Graaff</mark>, <mark>Jenny E. Greene</mark>

</div>
<div id="abstract">

**Abstract:** In November 2022, the\textit{James Webb Space Telescope}(\JWST) returned deep near-infrared images of Abell 2744 -- a powerful lensing cluster capable of magnifying distant, incipient galaxies beyond it. Together with the existing\textit{Hubble Space Telescope}(\HST) imaging, this publicly available dataset opens a fundamentally new discovery space to understand the remaining mysteries of the formation and evolution of galaxies across cosmic time. In this work, we detect and measure some 50,000 objects across the 45 arcmin$^2$\JWSTfootprint down to a$5 \sigma$limiting magnitude of$\sim$29.9 mag in 0.32$\arcsec$apertures. Photometry is performed using circular apertures on images matched to the point spread function of the reddest NIRCam band, F444W, and cleaned of bright cluster galaxies and the related intra-cluster light. To give an impression of the photometric performance, we measure photometric redshifts and achieve a$\sigma_{\rm NMAD}\approx0.03$based on known, but relatively small, spectroscopic samples. With this paper, we publicly release\HSTand\JWSTPSF-matched photometric catalogs optimized for bright and extended sources (0.7$\arcsec$apertures) and compact and faint sources (0.32$\arcsec$apertures) along with basic photometric redshifts, rest-frame colors, and individual magnification estimates.These catalogs will set the stage for efficient and deep spectroscopic follow-up of the first\JWST-selected samples in Summer 2023.

</div>

<div id="div_fig1">

<img src="tmp_2301.02671/./figures/dmag_Best-fit_0_32.png" alt="Fig13" width="100%"/>

**Figure 13. -** Photometry measured by AperPy in this work compared against predicted fluxes from the best-fit SED  from \eazy  in all 15 available bands. In each panel, the difference in AB magnitude  ($\Delta$ Mag) as a function of magnitude measured by AperPy is summarized by the log$_{10}$-scaled 2D grey density histogram. Colored curves indicate binned medians with two-sided envelopes enclosing 68\% of sources per bin. The overall median offset is labeled on each panel computed on all magnitudes up to the magnitude limit of the deepest 10\% of the corresponding image indicated by the colored solid vertical line; that of the median depth is indicated by the colored dashed vertical line. The typical photometric error derived by AperPy is indicated by the dotted grey curves.
     (*fig:dmag_model*)

</div>
<div id="div_fig2">

<img src="tmp_2301.02671/figures/detection_depth.png" alt="Fig9" width="100%"/>

**Figure 9. -** Schematic of depth variation across the noise-equalized LW coadd detection image aggregated from the DDT, GLASS, and UNCOVER programs. Combining F277W, F356W, and F444W, the 5 $\sigma$ depths in our LW detection band measured in 0.32$\arcsec$ apertures span $27-29$ mag. Poisson contributions of the brightest objects feature prominently in our \JWST weight maps. (*fig:detection_depths*)

</div>
<div id="div_fig3">

<img src="tmp_2301.02671/figures/badflag.png" alt="Fig11" width="100%"/>

**Figure 11. -** The selection of stars and spurious objects (bad pixels or model residuals) are determined from the flux ratio in large to small apertures above a limiting magnitude threshold. Stars that are unsaturated in \JWST are selected from the native resolution F200W image (blue), and co-added with \HST/F160W selected stars (yellow) and known GAIA stars (green). The LW images suffer from additional bad pixels and sources with contaminated photometry due to bad model residuals that are flagged (red).  Together, the bottom left panel shows that these flagged sources are well separated from the general galaxy population, with smaller values of F200W flux radius for a given magnitude.  Examples of point source postage stamps and bad pixels are shown at the top (left and right, respectively).  (*fig:badflag*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\um}{\mu\textnormal{m}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\um$}{\mu\textnormal{m}}$</div>



<div id="title">

#  NIRSpec phase curve for WASP-121b: dayside emission strongest eastward of the substellar point and nightside conditions conducive to cloud formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.03209-b31b1b.svg)](https://arxiv.org/abs/2301.03209)<mark>Appeared on: 2023-01-10</mark> - _Accepted for publication in Astrophysical Journal Letters on December 29, 2022_

</div>
<div id="authors">

<mark>Thomas Mikal-Evans</mark>, et al. -- incl., <mark>Jayesh M. Goyal</mark>, <mark>Nikole K. Lewis</mark>, <mark>Duncan A. Christie</mark>

</div>
<div id="abstract">

**Abstract:** We present the first exoplanet phase curve measurement made with the\textit{JWST}NIRSpec instrument, highlighting the exceptional stability of this newly-commissioned observatory for exoplanet climate studies. The target, WASP-121b, is an ultrahot Jupiter with an orbital period of 30.6 hr. We analyze two broadband light curves generated for the NRS1 and NRS2 detectors, covering wavelength ranges of 2.70-3.72$ $\um$$and 3.82-5.15$ $\um$$, respectively. Both light curves exhibit minimal systematics, with approximately linear drifts in the baseline flux level of 30 ppm/hr (NRS1) and 10 ppm/hr (NRS2). Assuming a simple brightness map for the planet described by a low-order spherical harmonic dipole, our light curve fits suggest that the phase curve peaks coincide with orbital phases$3.36 \pm 0.11 ^\circ$(NRS1) and$2.66 \pm 0.12 ^\circ$(NRS2) prior to mid-eclipse. This is consistent with the strongest dayside emission emanating from eastward of the substellar point. We measure planet-to-star emission ratios of$3,924 \pm 7  $ppm (NRS1) and$4,924 \pm 9  $ppm (NRS2) for the dayside hemisphere, and$136 \pm 8  $ppm (NRS1) and$630 \pm 10  $ppm (NRS2) for the nightside hemisphere. The latter nightside emission ratios translate to planetary brightness temperatures of$926 \pm 12  $K (NRS1) and$1,122 \pm 10  $K (NRS2), which are low enough for a wide range of refractory condensates to form, including enstatite and forsterite. A nightside cloud deck may be blocking emission from deeper, hotter layers of the atmosphere, potentially helping to explain why cloud-free 3D general circulation model simulations systematically over-predict the nightside emission for WASP-121b.

</div>

<div id="div_fig1">

<img src="tmp_2301.03209/./fig4_dayside_nightside_t.png" alt="Fig2" width="100%"/>

**Figure 2. -** Black diamonds show the measured dayside (top panel) and nightside (bottom panel) planet-to-star emission levels. \textit{Note that the measurement uncertainties are smaller than the diamond symbols.} Predictions from the cloud-free 3D GCM simulations of \citet{2018A&A...617A.110P} are also shown for heavy element enrichments of $1\times$ solar (blue lines) and $5\times$ solar (orange lines). Circle and square symbols show, respectively, the $1\times$ and $5\times$ solar GCM predictions binned to the light curve passbands, which are shown as green lines in the top panel. \label{fig:emission} (*fig:emission*)

</div>
<div id="div_fig2">

<img src="tmp_2301.03209/./fig3_model_parameters_t.png" alt="Fig4" width="100%"/>

**Figure 4. -** Dark blue histograms show posterior distributions for model parameters that were varied in the light curve fitting. Light blue histograms show corresponding distributions for other parameters that were subsequently derived. Posterior medians and $\pm 34$\% credible intervals are listed in black font. Red dashed lines show the priors. For the normal priors, the mean $\mu$ and standard deviation $\sigma$ are listed in red font. Half-normal priors were adopted for the $\sigma_{\rm{syst}}$ parameters, allowing only positive values and with scale $\sigma$ listed in red font. Vertical axes are not labeled, as arbitrary normalizations have been applied to all plotted distributions. \label{fig:pars} (*fig:pars*)

</div>
<div id="div_fig3">

<img src="tmp_2301.03209/./fig2_light_curves_t.png" alt="Fig3" width="100%"/>

**Figure 3. -** \textit{Panels (a) and (b):} Raw broadband light curves for the NRS1 and NRS2 detectors, respectively. Orange lines show the best-fit light curve models. Blue lines show the corresponding instrument baseline trends. Grey horizontal lines are calibrated to the bottoms of the first eclipse, to highlight the instrumental drift over the course of the observation. \textit{Panels (c) and (d):} The same light curves focusing on the planetary emission signal and after correcting for the instrument baseline trends shown by the blue lines in panels (a) and (b). \textit{Panels (e) and (f):} Black points show the residuals between the data and best-fit models for the NRS1 and NRS2 detectors, respectively. Orange lines show the residuals after applying a Gaussian filter to smooth the random noise. Green and purple shading show the times of transit and eclipse, respectively, with darker ranges corresponding to ingress and egress times. \label{fig:lc} (*fig:lc*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}[1]{\SI[mode=text]{#1}{\kilo\meter\per\second}}$
$\newcommand{\ghz}[1]{\SI{#1}{\giga\hertz}}$
$\newcommand{\au}{\si{\astronomicalunit}}$
$\newcommand{\sci}[2]{\ensuremath{#1\times10^{#2}}}$
$\newcommand{\sectionautorefname}{Section}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}[1]{\SI[mode=text]{#1}{\kilo\meter\per\second}}$
$\newcommand{$\ghz$}[1]{\SI{#1}{\giga\hertz}}$
$\newcommand{$\au$}{\si{\astronomicalunit}}$
$\newcommand{$\sci$}[2]{$\ensuremath${#1\times10^{#2}}}$
$\newcommand{$\sectionautorefname$}{Section}$</div>



<div id="title">

# An APEX study of molecular outflows in FUor-type stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.03387-b31b1b.svg)](https://arxiv.org/abs/2301.03387)<mark>Appeared on: 2023-01-10</mark> - _Accepted for publication in ApJ. 29 pages, 8 tables, 10 figures, 2 figuresets_

</div>
<div id="authors">

Fernando Cruz-S\'aenz de Miera, et al. -- incl., <mark>Thomas Henning</mark>

</div>
<div id="abstract">

**Abstract:** FU Orionis-type objects (FUors) are low-mass pre-main-sequence objects which go through a short-lived phase ($\sim$100 years) of increased mass accretion rate (from 10$^{-8}$to 10$^{-4}$M$_\odot$yr$^{-1}$).These eruptive young stars are in the early stages of stellar evolution and, thus, still deeply embedded in a massive envelope that feeds material to the circumstellar disk that is then accreted onto the star.Some FUors drive molecular outflows, i.e. low-velocity wide-angle magneto-hydrodynamical winds, that inject energy and momentum back to the surrounding envelopes, and help clear the material surrounding the young star.Here we present a$^{12}$CO (3--2),$^{13}$CO (3--2) and$^{12}$CO (4--3) survey of 20 FUor-type eruptive young stars observed with APEX\@.We use our$^{13}$CO (3--2) observations to measure the masses of the envelopes surrounding each FUor and find an agreement with the FUor evolutionary trend found from the\SI{10}{\micro\meter}silicate feature.We find outflows in 11 FUors, calculate their masses and other kinematic properties, and compare these with those of outflows found around quiescent young stellar objects gathered from the literature.This comparison indicates that outflows in FUors are more massive than outflows in quiescent sources, and that FUor outflows have a higher ratio outflow mass with respect to the envelope than the quiescent sample, indicating that the eruptive young stars have lower star-forming efficiencies.Finally, we found that the outflow forces in FUors are similar to those of quiescent young stellar objects, indicating that their accretion histories are similar or that the FUor outflows have lower velocities.

</div>

<div id="div_fig1">

<img src="tmp_2301.03387/figures/L1551IRS5_parabolas.png" alt="Fig12" width="100%"/>

**Figure 12. -** Optical depth correction for L1551 IRS 5.
  In the left panel we show the line profiles, where the green, purple and black colors represent the $^{13}$CO, the observed $^{12}$CO and the corrected $^{12}$CO, respectively, and the vertical dashed lines indicate the rage of velocities used in the parabola fit.
  The right panel shows the ratio of main beam temperatures (T$_\mathrm{MB}$), where the light blue crosses are all the values of the ratio for each velocity channel, in pink dots with errorbars are the points used in the parabolic fitting, and the black line is the resulting best-fitted parabola. The complete figure set (10 images) is available in the online journal.\label{fig:correction_l1551irs5} (*fig:correction_l1551irs5*)

</div>
<div id="div_fig2">

<img src="tmp_2301.03387/figures/line_profiles.png" alt="Fig6" width="100%"/>

**Figure 6. -** CO line profiles of our targets observed with APEX\@. The vertical dotted line is the systemic velocity. The vertical dashed lines are the range of velocities of the CO (3--2) outflows. The line profiles have been smoothed for presentation purposes.\label{fig:spectra} (*fig:spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2301.03387/figures/outflows_32.png" alt="Fig7" width="100%"/>

**Figure 7. -** The red and blue contours show redshifted and blueshifted CO (J=3--2) emission integrated in the velocity ranges indicated in $\au$toref{fig:spectra}.
The star symbols mark the stellar position as given in $\au$toref{tab:values}.
The hatched circle in the bottom right frame is the APEX beam size and the arrows indicate the orientation of the outflow.
In the cases of V2775 Ori and V960 Mon, the outflow appears to be expanding in the direction of the line of sight.
For L1551 IRS 5, the blueshifted contours are 3, 13, 24, 35, 45, 56, 67 and 78$\upsigma$ for $\upsigma$=\SI{0.38}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 18, 33, 48, 63, 78, 93 and 109$\upsigma$ for $\upsigma$=\SI{0.33}{\kelvin\kilo\meter\per\second}.
For Haro 5a IRS, the blueshifted contours are 3, 7, 12, 16, 21, 25, 30 and 35$\upsigma$ for $\upsigma$=\SI{0.61}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 6, 10, 14, 18, 22, 26 and 30$\upsigma$ for $\upsigma$=\SI{0.48}{\kelvin\kilo\meter\per\second}.
For Reipurth 50, the blueshifted contours are 3, 5, 7, 9, 11, 13, 15 and 17$\upsigma$ for $\upsigma$=\SI{0.48}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 6, 10, 14, 17, 21, 25 and 29$\upsigma$ for $\upsigma$=\SI{0.43}{\kelvin\kilo\meter\per\second}.
For V2775 Ori, the blueshifted contours are 3, 12, 22, 31, 41, 50, 60 and 70$\upsigma$ for $\upsigma$=\SI{0.73}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 7, 12, 17, 21, 26, 31 and 36$\upsigma$ for $\upsigma$=\SI{0.68}{\kelvin\kilo\meter\per\second}.
For V899 Mon, the blueshifted contours are 3, 5, 7, 9, 11, 13, 15 and 17$\upsigma$ for $\upsigma$=\SI{0.52}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 4, 6, 8, 9, 11, 13 and 15$\upsigma$ for $\upsigma$=\SI{0.43}{\kelvin\kilo\meter\per\second}.
For V900 Mon, the blueshifted contours are 3, 4, 5, 6, 7, 8, 9 and 10$\upsigma$ for $\upsigma$=\SI{0.62}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 4, 5, 6, 7 and 8$\upsigma$ for $\upsigma$=\SI{0.47}{\kelvin\kilo\meter\per\second}.
For V960 Mon, the blueshifted contours are 3, 6, 9, 13, 16, 20, 23 and 27$\upsigma$ for $\upsigma$=\SI{0.44}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 7, 11, 15, 19, 23, 27 and 32$\upsigma$ for $\upsigma$=\SI{0.25}{\kelvin\kilo\meter\per\second}.
For Z CMa, the blueshifted contours are 3, 4, 6, 8, 9, 11, 13 and 15$\upsigma$ for $\upsigma$=\SI{0.52}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 8, 13, 19, 24, 30, 35 and 41$\upsigma$ for $\upsigma$=\SI{0.39}{\kelvin\kilo\meter\per\second}.
For iPTF 15afq, the blueshifted contours are 3, 5, 7, 9, 11, 13, 15 and 17$\upsigma$ for $\upsigma$=\SI{0.44}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 6, 10, 13, 17, 20, 24 and 28$\upsigma$ for $\upsigma$=\SI{0.32}{\kelvin\kilo\meter\per\second}.
For GM Cha, the blueshifted contours are 3, 5, 7, 9, 11, 13, 15 and 18$\upsigma$ for $\upsigma$=\SI{0.43}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 9, 16, 23, 29, 36, 43 and 50$\upsigma$ for $\upsigma$=\SI{0.39}{\kelvin\kilo\meter\per\second}.
For V346 Nor, the blueshifted contours are 3, 12, 22, 32, 41, 51, 61 and 71$\upsigma$ for $\upsigma$=\SI{0.52}{\kelvin\kilo\meter\per\second} while the redshifted contours are 3, 11, 20, 29, 38, 47, 56 and 65$\upsigma$ for $\upsigma$=\SI{0.56}{\kelvin\kilo\meter\per\second}.
\label{fig:outflows} (*fig:outflows*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

161  publications in the last 7 days.
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers